In [1]:
using Revise

In [2]:
using MeshCatMechanisms
using MeshCat
using RigidBodyDynamics
using Gurobi

In [3]:
import LCPSim
import LearningMPC
import BoxValkyries
import Nets
reload("LearningMPC")
reload("BoxValkyries")

In [4]:
robot = BoxValkyries.BoxValkyrie();

In [5]:
mvis = MechanismVisualizer(robot)
IJuliaCell(mvis)

Listening on 127.0.0.1:7001...
zmq_url=tcp://127.0.0.1:6001
web_url=http://127.0.0.1:7001/static/


MeshCat.IJuliaCell(MeshCat.ViewerWindow(ZMQ.Context(Ptr{Void} @0x0000000005c24c50, ZMQ.Socket[ZMQ.Socket(Ptr{Void} @0x0000000005c44160, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000005c6a840, 57, (1, 0), Condition(Any[]), (false, false), 0))]), ZMQ.Socket(Ptr{Void} @0x0000000005c44160, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000005c6a840, 57, (1, 0), Condition(Any[]), (false, false), 0)), "http://127.0.0.1:7001/static/", "tcp://127.0.0.1:6001", MeshCat.ZMQServer.ZMQWebSocketBridge(ip"127.0.0.1", Set{WebSockets.WebSocket}(), MeshCat.SceneTrees.SceneNode(Nullable{Array{UInt8,1}}(), Nullable{Array{UInt8,1}}(), Dict("meshcat"=>MeshCat.SceneTrees.SceneNode(#NULL, #NULL, Dict("robot"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x72, 0x6f, 0x62, 0x6f, 0x74], Dict("world"=>MeshCat.SceneTrees.SceneNode(#NULL, #NULL, Dict("dummy"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x6c, 0x64, 0x2f, 0x64, 0x75, 0x6d, 0x6d, 0x79], Dict("base"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x64, 0x75, 0x6d, 0x6d, 0x79, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict("core"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x62, 0x61, 0x73, 0x65, 0x2f, 0x63, 0x6f, 0x72, 0x65], Dict("rh_dummy"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x2f, 0x72, 0x68, 0x5f, 0x64, 0x75, 0x6d, 0x6d, 0x79], Dict("after_core_to_rh_rotation"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x5f, 0x72, 0x6f, 0x74, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict("geometry_2"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String,MeshCat.SceneTrees.SceneNode}()))),"rh"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x5f, 0x64, 0x75, 0x6d, 0x6d, 0x79, 0x2f, 0x72, 0x68], Dict("after_core_to_rh_extension"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x65, 0x78, 0x74, 0x65, 0x6e, 0x73, 0x69, 0x6f, 0x6e], Dict("geometry_3"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String,MeshCat.SceneTrees.SceneNode}()))))))),"rf_dummy"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x2f, 0x72, 0x66, 0x5f, 0x64, 0x75, 0x6d, 0x6d, 0x79], Dict("after_core_to_rf_rotation"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x5f, 0x72, 0x6f, 0x74, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict("geometry_6"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x36], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x36], Dict{String,MeshCat.SceneTrees.SceneNode}()))),"rf"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x5f, 0x64, 0x75, 0x6d, 0x6d, 0x79, 0x2f, 0x72, 0x66], Dict("after_core_to_rf_extension"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x65, 0x78, 0x74, 0x65, 

In [14]:
xstar = BoxValkyries.nominal_state(robot)

mpc_params = LearningMPC.MPCParams(
    Δt=0.05,
    horizon=10,
    mip_solver=GurobiSolver(Gurobi.Env(), OutputFlag=1, 
        TimeLimit=120, 
        MIPGap=1e-1, 
        MIPGapAbs=5e-1,
        FeasibilityTol=1e-3),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

Q, R = BoxValkyries.default_costs(robot)
feet = findbody.(robot.mechanism, ["lf", "rf"])
lqrsol = LearningMPC.LQRSolution(xstar, Q, R, mpc_params.Δt, Point3D.(default_frame.(feet), 0., 0., 0.))
lqrsol.S .= 1 ./ mpc_params.Δt .* Q

mpc_controller = LearningMPC.MPCController(robot.mechanism, 
    robot.environment, mpc_params, lqrsol, 
    [lqrsol]);

sample_sink = LearningMPC.MPCSampleSink{Float64}(true)
playback_sink = LearningMPC.PlaybackSink(mvis, mpc_params.Δt)

mpc_controller.callback = LearningMPC.call_each(
    sample_sink,
    playback_sink,
)


Academic license - for non-commercial use only
Academic license - for non-commercial use only


(::#35) (generic function with 1 method)

In [16]:
x0 = BoxValkyries.nominal_state(robot)
set_velocity!(x0, velocity(x0) .+ 2 .* randn(length(velocity(x0))))
mpc_controller(x0)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 1960 rows, 941 columns and 6340 nonzeros
Model has 410 quadratic objective terms
Variable types: 581 continuous, 360 integer (360 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+03]
  Objective range  [9e-19, 4e+02]
  QObjective range [2e-03, 4e+02]
  Bounds range     [5e-02, 1e+03]
  RHS range        [1e-16, 1e+03]

Loaded MIP start with objective 255.884

Presolve removed 989 rows and 115 columns
Presolve time: 0.01s
Presolved: 971 rows, 826 columns, 3767 nonzeros
Presolved model has 382 quadratic objective terms
Variable types: 514 continuous, 312 integer (312 binary)

Root relaxation: objective 1.949537e+01, 1389 iterations, 0.07 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   19.49537    0  146  255.88406   19.49537  92.4%     -    0s
     0   

10-element Array{Float64,1}:
   0.0      
   0.0      
   2.0      
   0.0892927
   4.00603  
  -5.42407  
  -2.0      
   2.0      
 -15.0594   
 -12.5368   

In [12]:
LearningMPC.playback(mvis, playback_sink.last_trajectory, mpc_params.Δt)

length(ts) = 10
length(results) = 10


In [29]:
x0 = BoxValkyries.nominal_state(robot)
set_velocity!(x0, velocity(x0) .+ 8 .* (rand(length(velocity(x0))) .- 0.5))
results = LCPSim.simulate(x0, mpc_controller, robot.environment, 0.05, 20, mpc_params.lcp_solver)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 1960 rows, 941 columns and 6340 nonzeros
Model has 410 quadratic objective terms
Variable types: 581 continuous, 360 integer (360 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+03]
  Objective range  [9e-19, 4e+02]
  QObjective range [2e-03, 4e+02]
  Bounds range     [5e-02, 1e+03]
  RHS range        [3e-03, 1e+03]

Loaded MIP start with objective 2317.19

Presolve removed 979 rows and 106 columns
Presolve time: 0.01s
Presolved: 981 rows, 835 columns, 3802 nonzeros
Presolved model has 383 quadratic objective terms
Variable types: 521 continuous, 314 integer (314 binary)

Root relaxation: objective 8.186277e+01, 1592 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   81.86277    0  154 2317.19188   81.86277  96.5%     -    0s
     0   

20-element Array{LCPSim.LCPUpdate{Float64,Float64,Float64},1}:
 LCPSim.LCPUpdate{Float64,Float64,Float64}(LCPSim.StateRecord{Float64,Float64}(Spanning tree:
Vertex: world (root)
  Vertex: dummy, Edge: base_x
    Vertex: base, Edge: base_z
      Vertex: core, Edge: core_to_world
        Vertex: rh_dummy, Edge: core_to_rh_rotation
          Vertex: rh, Edge: core_to_rh_extension
        Vertex: lh_dummy, Edge: core_to_lh_rotation
          Vertex: lh, Edge: core_to_lh_extension
        Vertex: rf_dummy, Edge: core_to_rf_rotation
          Vertex: rf, Edge: core_to_rf_extension
        Vertex: lf_dummy, Edge: core_to_lf_rotation
          Vertex: lf, Edge: core_to_lf_extension
No non-tree joints., [0.117889, 1.17473, -0.035626, 0.09697, -0.05, -0.05, 0.627828, 0.744677, 0.885576, 0.56568, 2.35778, 2.49459, -0.712521, 1.9394, -1.0, -1.0, 2.55656, 4.89353, 1.71152, -4.68641], [0.117889, 1.17473, -0.035626, 0.09697, -0.05, -0.05, 0.627828, 0.744677, 0.885576, 0.56568], [2.35778, 2.49459, -0.

In [50]:
LearningMPC.playback(mvis, results, 0.05)

length(ts) = 20
length(results) = 20


In [49]:
set_configuration!(mvis, configuration(x0))

In [11]:
results = LCPSim.simulate(x0, x -> zeros(velocity(x)), robot.environment, 0.01, 1, mpc_params.lcp_solver)
set_configuration!(x0, configuration(results[end].state))
set_velocity!(x0, velocity(results[end].state))
set_configuration!(mvis, configuration(x0))